COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: Santiago Sanchez
#### Date: Jan 12 2026

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [47]:
import json, requests
from datetime import date, datetime, timedelta
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First, a short reminder about python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [48]:
today = date.today()   # get a date object representing today's date
print(today, type(today))
# f"..."  is a format string - notice how the variable `today` is formatted into the string
print(f'Today is: {today}')

2026-01-12 <class 'datetime.date'>
Today is: 2026-01-12


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [71]:
NASA_NEO_FEED_API_URL = "https://api.nasa.gov/neo/rest/v1/feed"
API_KEY = 'R3wOZGJTgZAFT8rAXW6dXBVStgToNn2csVUfD19k'  # substitute your API key here

start_date = str(date.today())  #  Future enhancement:  str(date.today())   # Today's date as a string!
end = str(date.today())

feed_params = dict(
    start_date=start_date,
    end_date=end,
    api_key=API_KEY,
)

response = requests.request("GET", NASA_NEO_FEED_API_URL, params=feed_params)
response.raise_for_status()  # raise an error if request is not successful

feed_data = json.loads(response.text)  # recall json.loads from lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = feed_data['element_count']
neos = feed_data['near_earth_objects'][start_date]
print(f'{n_results} Near Earth Objects found for {start_date}')
pprint(neos[:2])
print(feed_data.keys())

17 Near Earth Objects found for 2026-01-12
[{'absolute_magnitude_h': 18.12,
  'close_approach_data': [{'close_approach_date': '2026-01-12',
                           'close_approach_date_full': '2026-Jan-12 10:26',
                           'epoch_date_close_approach': 1768213560000,
                           'miss_distance': {'astronomical': '0.0825924422',
                                             'kilometers': '12355653.431218114',
                                             'lunar': '32.1284600158',
                                             'miles': '7677447.0342094132'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '51900.5564593488',
                                                 'kilometers_per_second': '14.4168212387',
                                                 'miles_per_hour': '32248.9874288659'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 4634.7363067389,
    

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [75]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified on {start_date}.')
print(type(hazards))
print(hazards)

1 potentially hazardous asteroids identified on 2026-01-12.
<class 'list'>
[{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2242708?api_key=R3wOZGJTgZAFT8rAXW6dXBVStgToNn2csVUfD19k'}, 'id': '2242708', 'neo_reference_id': '2242708', 'name': '242708 (2005 UK1)', 'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2242708', 'absolute_magnitude_h': 18.12, 'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.6317641482, 'estimated_diameter_max': 1.4126675811}, 'meters': {'estimated_diameter_min': 631.7641481849, 'estimated_diameter_max': 1412.6675810887}, 'miles': {'estimated_diameter_min': 0.3925599205, 'estimated_diameter_max': 0.8777906675}, 'feet': {'estimated_diameter_min': 2072.7170879309, 'estimated_diameter_max': 4634.7363067389}}, 'is_potentially_hazardous_asteroid': True, 'close_approach_data': [{'close_approach_date': '2026-01-12', 'close_approach_date_full': '2026-Jan-12 10:26', 'epoch_date_close_approach': 1768213560000, 'relative_veloci

## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [76]:
# Ex. 1 your code here
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified on {start_date}.')

1 potentially hazardous asteroids identified on 2026-01-12.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [77]:
NASA_NEO_API_URL = "https://api.nasa.gov/neo/rest/v1/neo/{id}"
neo_params = dict(
    api_key=API_KEY,
)
hazard_id = hazards[0]['id']
print(hazard_id)

response = requests.request("GET", NASA_NEO_API_URL.format(id=hazard_id), params=neo_params)
response.raise_for_status()  # raise an error if request is not successful

neo_data = json.loads(response.text)

print(f'NEO {neo_data["name"]} with {len(neo_data["close_approach_data"])} approach records.  {"Hazardous." if neo_data["is_potentially_hazardous_asteroid"] else ""}')
print(f'Close approach records:')
close_approach_data = neo_data['close_approach_data']
pprint(close_approach_data)

2242708
NEO 242708 (2005 UK1) with 18 approach records.  Hazardous.
Close approach records:
[{'close_approach_date': '1927-05-19',
  'close_approach_date_full': '1927-May-19 07:18',
  'epoch_date_close_approach': -1345048920000,
  'miss_distance': {'astronomical': '0.0681969234',
                    'kilometers': '10202114.481193158',
                    'lunar': '26.5286032026',
                    'miles': '6339299.9813672604'},
  'orbiting_body': 'Earth',
  'relative_velocity': {'kilometers_per_hour': '67905.4268272428',
                        'kilometers_per_second': '18.8626185631',
                        'miles_per_hour': '42193.7914638498'}},
 {'close_approach_date': '1931-04-16',
  'close_approach_date_full': '1931-Apr-16 10:55',
  'epoch_date_close_approach': -1221656700000,
  'miss_distance': {'astronomical': '0.1719506929',
                    'kilometers': '25723457.402864123',
                    'lunar': '66.8888195381',
                    'miles': '15983815.2507779774

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


Pseudocode:
need to create a function that takes a list, with the close approach data, and returns a 2-tuple with (the date that happen, the distance of miss km)

parameters:

close_approach_date

miss_distance

use the [(approach[close_approach_date], approach[miss_distance]) for approach in close_approach_data ]
example [0] should be (1927-05-19, 10202114.481193158)

In [79]:
# Ex. 2 your code here

def approaches_data(my_list):
  '''Returns a the the closest approach as a tuple '''

  approaches = [(approach["close_approach_date"], approach["miss_distance"]['kilometers']) for approach in my_list if approach["orbiting_body"] == "Earth"] #Get all the approaches as a tuple
  closest_approach = approaches[0] #set the closetw

  # return tuple(approach for approach in approaches if float(approach[1]) < float(closest_approach[1])) #Create Tuple using list comprehension

  for approach in approaches:
    if float(approach[1]) < float(closest_approach[1]):
      closest_approach = approach

  return closest_approach

data = approaches_data(close_approach_data)
print(data)


('2188-05-19', '8897211.366066376')


## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, when it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [54]:
# Ex. 3 (challenge) your code here
delta = timedelta(weeks = 1)
print(delta)

7 days, 0:00:00
